In [2]:
%matplotlib inline

In [294]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy.io import loadmat
from scipy.optimize import minimize
from sklearn.linear_model import LogisticRegression

In [242]:
data = loadmat('machine-learning-ex3/ex3/ex3data1.mat')

In [243]:
data.keys()

['y', 'X', '__version__', '__header__', '__globals__']

In [244]:
X = np.c_[np.ones((data['X'].shape[0],1)), data['X']]
y = data['y']

In [245]:
X.shape, y.shape, np.unique(y), np.unique(X)

((5000, 401),
 (5000, 1),
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=uint8),
 array([-0.13196323, -0.11993388, -0.11220256, ...,  1.12024444,
         1.12393404,  1.1276883 ]))

In [246]:
def sigmoid(z):
    return 1/(1 + np.exp(-z))

In [285]:
def lr_cost_function_with_reg(theta, reg, x, y):
    m = y.size
    h = sigmoid(X.dot(theta))
    cost = -1*(1.0/m)*(np.log(h).T.dot(y)+np.log(1-h).T.dot(1-y)) + (reg/(2*m))*np.sum(np.square(theta[1:]))
    return(cost[0]) 

In [306]:
def lr_gradient_with_reg(theta, reg, x, y):
    m = y.size
    h = sigmoid(X.dot(theta.reshape(-1,1)))
    grad = (1.0/m)*X.T.dot(h-y) + (reg/m)*np.r_[[[0]],theta[1:].reshape(-1,1)]
    return grad.flatten()

### One-vs-all Classification

In [287]:
def one_vs_all(features, classes, n_labels, reg):
    initial_theta = np.zeros((X.shape[1],1))
    all_theta = np.zeros((n_labels, X.shape[1]))
    for c in np.arange(1, n_labels+1):
        res = minimize(lr_cost_function_with_reg, initial_theta, args=(reg, features, (classes == c)*1), method=None,
                       jac=lr_gradient_with_reg, options={'maxiter':50})
        all_theta[c-1] = res.x
    return all_theta

In [288]:
theta = one_vs_all(X, y, 10, 0.1)
print theta

[[ -2.39142338e+00   0.00000000e+00   0.00000000e+00 ...,   6.90403614e-04
    1.25548916e-07   0.00000000e+00]
 [ -3.01137869e+00   0.00000000e+00   0.00000000e+00 ...,   1.48908371e-03
   -1.69495224e-04   0.00000000e+00]
 [ -4.51398334e+00   0.00000000e+00   0.00000000e+00 ...,  -1.60669585e-05
    3.43368479e-07   0.00000000e+00]
 ..., 
 [ -8.41428002e+00   0.00000000e+00   0.00000000e+00 ...,  -6.96539602e-05
    6.26414118e-06   0.00000000e+00]
 [ -5.13253825e+00   0.00000000e+00   0.00000000e+00 ...,  -1.54282889e-04
    5.92217956e-06   0.00000000e+00]
 [ -4.38685169e+00   0.00000000e+00   0.00000000e+00 ...,  -3.60354278e-04
    9.53842298e-06   0.00000000e+00]]


### One-vs-all Prediction

In [289]:
def predict_one_vs_all(all_theta, features):
    probs = sigmoid(X.dot(all_theta.T))
    print probs
        
    # Adding one because Python uses zero based indexing for the 10 columns (0-9),
    # while the 10 classes are numbered from 1 to 10.
    return(np.argmax(probs, axis=1)+1)

In [290]:
pred = predict_one_vs_all(theta, X)
print pred.shape

[[  1.13808117e-13   2.53747146e-04   3.73214932e-05 ...,   2.20791707e-05
    1.81701529e-04   9.99679426e-01]
 [  8.19183673e-12   2.01268865e-05   9.73072658e-05 ...,   1.03747290e-06
    6.77071791e-06   9.99705449e-01]
 [  2.85590783e-15   1.38351748e-04   6.38321914e-04 ...,   2.35529203e-02
    1.80781339e-03   9.97154524e-01]
 ..., 
 [  4.41544246e-02   1.60737199e-03   3.27288656e-02 ...,   1.37621224e-03
    6.12110061e-01   2.31664369e-09]
 [  7.08546286e-12   3.99341869e-08   1.01698252e-06 ...,   1.13880413e-01
    9.35541108e-01   2.98405727e-08]
 [  2.56192943e-16   7.82747234e-05   1.32053939e-09 ...,   2.24390446e-03
    2.55572306e-01   2.22769196e-03]]
(5000,)


In [291]:
print (pred == y.ravel()).mean()

0.9324


### Multi-class Classification with Scikit Learn

In [301]:
Logit = LogisticRegression(penalty='l2', C=10, solver='newton-cg', fit_intercept=False, max_iter=50)
Logit.fit(X,y.ravel())

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=False,
          intercept_scaling=1, max_iter=50, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='newton-cg', tol=0.0001,
          verbose=0, warm_start=False)

In [302]:
np.mean(Logit.predict(X)==y.ravel())

0.96499999999999997

## Neural Networks

In [303]:
weights = loadmat('machine-learning-ex3/ex3/ex3weights.mat')

In [304]:
weights.keys()

['Theta2', '__version__', '__header__', 'Theta1', '__globals__']

In [305]:
theta1 = weights["Theta1"]
theta2 = weights["Theta2"]
print theta1.shape, theta2.shape

(25, 401) (10, 26)


In [311]:
def predict(theta_1, theta_2, features):
    z2 = theta_1.dot(features.T)
    a2 = np.c_[np.ones((data['X'].shape[0],1)), sigmoid(z2).T]
    z3 = theta2.dot(a2.T)
    a3 = sigmoid(z3).T
    return np.argmax(a3,axis=1) + 1

In [312]:
np.mean(predict(theta1, theta2, X) == y.ravel())

0.97519999999999996